In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import warnings
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")
from tpot import TPOTClassifier
import glob
print(os.listdir('../input/daily-rpi-2019'))

['WK_Col_02_25_2019.csv', 'WK_Col_03_15_2019.csv', 'WK_Col_02_24_2019.csv', 'WK_Col_02_16_2019.csv', 'WK_Col_03_18_2019.csv', 'WK_Col_03_17_2019.csv', 'WK_Col_02_15_2019.csv', 'WK_Col_03_08_2019.csv', 'WK_Col_02_17_2019.csv', 'WK_Col_03_11_2019.csv', 'WK_Col_03_05_2019.csv', 'full_team_crosswalk.csv', 'WK_Col_02_19_2019.csv', 'teams_to_rpi_teams.csv', 'WK_Col_Aggregate.csv', 'WK_Col_03_12_2019.csv', 'train_master.csv', 'WK_Col_03_04_2019.csv', 'WK_Col_03_14_2019.csv', 'test_stage1_Prepped.csv', 'WK_Col_01_28_2019.csv', 'reg_season_tourney_output_all.csv', 'WK_Col_02_28_2019.csv', 'WK_Col_02_20_2019.csv', 'SampSubmishStage1.csv', 'WK_Col_03_06_2019.csv', 'WK_Col_03_07_2019.csv', 'Stage2_LR_AYX_Predictions.csv', 'WK_Col_02_13_2019.csv', 'WK_Col_03_01_2019.csv', 'WK_Col_03_02_2019.csv', 'WK_Col_02_26_2019.csv', 'WK_Col_03_13_2019.csv', 'train_stage2.csv', 'WK_Col_02_27_2019.csv', 'WK_Col_02_22_2019.csv', 'SampSubmishStage1_Prepped.csv', 'WK_Col_02_23_2019.csv', 'train_stage1_Prepped.csv',

Previously (during stage 1) prepped data for using a variety of model evaluation techniques. 

In [9]:
st1_tr =  pd.read_csv("../input/daily-rpi-2019/train_stage1_Prepped.csv")
st1_tr.head()
st1_tt =  pd.read_csv("../input/daily-rpi-2019/test_stage1_Prepped.csv")
st1_tt.head()

,PPG_1,OPPG_1,PPG_diff,TOPG_1,FG3_PCT_1,AOE_1,ADE_1,AEE_1,PPG_2,OPPG_2,TOPG_2,FG3_PCT_2,AOE_2,ADE_2,AEE_2,seed_diff,OPPG_diff,AOE_diff,ADE_diff,AEE_diff,TOPG_diff,FG3_PCT_diff,target
0,72.166667,60.722222,0.666667,13.00,0.434109,1.158064,1.030348,2.188413,71.500000,57.687500,11.187500,0.368852,1.160307,1.063350,2.223656,15,3.034722,-0.002242,-0.033001,-0.035243,1.812500,0.065256,0
1,72.166667,60.722222,-10.083333,13.00,0.434109,1.158064,1.030348,2.188413,82.250000,72.562500,11.687500,0.401146,1.210949,0.932917,2.143866,0,-11.840278,-0.052884,0.097431,0.044547,1.312500,0.032962,1
2,67.500000,54.350000,-7.884615,13.95,0.398058,1.120207,1.098966,2.219174,75.384615,62.076923,8.000000,0.390110,1.226225,0.992088,2.218313,13,-7.726923,-0.106017,0.106878,0.000861,5.950000,0.007948,0
3,74.400000,56.966667,-3.488889,10.30,0.366292,1.187905,1.094870,2.282775,77.888889,62.592593,11.000000,0.411157,1.200499,1.035152,2.235651,-7,-5.625926,-0.012594,0.059719,0.047125,-0.700000,-0.044865,1
4,74.400000,56.966667,4.214815,10.30,0.366292,1.187905,1.094870,2.282775,70.185185,56.814815,9.814815,0.347480,1.120725,1.090824,2.211548,-3,0.151852,0.067181,0.004046,0.071227,0.485185,0.018812,1


Processed data for Stage 2 - both the training file of historical tournament data for the period of 2014-2018 containing all NCAA results. And the "test" file which is this year's Stage 2 submission file with the processed team level data ready to be predicted on. 

In [10]:
st2_tr =  pd.read_csv("../input/daily-rpi-2019/train_stage2.csv")
st2_tr.head()




,seed_diff,PPG_diff,OPPG_diff,AOE_diff,ADE_diff,AEE_diff,TOPG_diff,FG3_PCT_diff,target
0,3,-0.744118,-1.370588,-0.041326,0.045585,0.004259,2.258824,-0.032315,1
1,-8,7.721739,4.615652,-0.002560,0.020684,0.018124,0.017391,-0.018063,0
2,-1,0.678261,5.398261,-0.026397,-0.049631,-0.076028,0.278261,0.007224,1
3,-4,4.427273,3.010909,0.007776,-0.000814,0.006962,1.709091,-0.026811,0
4,-15,14.410526,9.183158,0.031702,0.007947,0.039649,1.326316,-0.000630,0


In [11]:
st2_tt =  pd.read_csv("../input/daily-rpi-2019/test_stage2.csv")
st2_tt.head()

,ID,Team1,T1_RPI,Seed_1,Team2,T2_RPI,Seed_2,seed_diff,PPG_diff,OPPG_diff,AOE_diff,ADE_diff,AEE_diff,TOPG_diff,FG3_PCT_diff
0,2019_1101_1113,1101,ABL CHRISTIAN,15,1113,ARIZONA ST,11,4,-6.08836,-8.16595,0.003957,0.033703,0.037660,-1.93858,0.042035
1,2019_1101_1120,1101,ABL CHRISTIAN,15,1120,AUBURN,5,10,-7.15821,-3.69168,-0.058369,0.012063,-0.046306,-0.49189,0.002446
2,2019_1113_1120,1113,ARIZONA ST,11,1120,AUBURN,5,6,-1.06985,4.47427,-0.062326,-0.021640,-0.083966,1.44669,-0.039589
3,2019_1101_1124,1101,ABL CHRISTIAN,15,1124,BAYLOR,9,6,0.06789,-2.29095,-0.000515,0.036455,0.035940,-1.62608,0.043639
4,2019_1113_1124,1113,ARIZONA ST,11,1124,BAYLOR,9,2,6.15625,5.87500,-0.004472,0.002752,-0.001720,0.31250,0.001604


Now let's try to find a good model for evaluating this training data. Let's first separate the features (predictor variables) from the target (outcome or dependent) variable. Let's also prepare an autoML library for analyzing this data. And let's also split the model for train and test data. 

In [13]:
features = ['seed_diff', 'PPG_diff', 'OPPG_diff', 'AOE_diff', 'ADE_diff', 'AEE_diff', 'TOPG_diff', 'FG3_PCT_diff']
X = st2_tr[features]
y = st2_tr['target']

# split model for train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=7)

#pipeline_optimizer = TPOTClassifier()
#pipeline_optimizer = TPOTClassifier(generations=25, population_size=250, scoring='neg_log_loss', cv=8,
#                                    random_state=42, verbosity=2)
#pipeline_optimizer.fit(X_train, y_train)
#print(pipeline_optimizer.score(X_test, y_test))
# 0.7099236641221374
#pipeline_optimizer.export('tpot_exported_pipeline.py')

# Best pipeline according to above iteration of TPOT: LogisticRegression(LinearSVC(input_matrix, C=5.0, dual=True, loss=hinge, penalty=l2, tol=0.001), C=10.0, dual=False, penalty=l1)

#pipeline_optimizer = TPOTClassifier()
#pipeline_optimizer = TPOTClassifier(generations=100, population_size=100, scoring="neg_log_loss", cv=5,
#                                    random_state=42, verbosity=2)
#pipeline_optimizer.fit(X_train, y_train)
#print(pipeline_optimizer.score(X_test, y_test))

from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.preprocessing import Normalizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression

#LogisticRegression(LinearSVC(input_matrix, C=5.0, dual=True, loss=hinge, penalty=l2, tol=0.001), C=10.0, dual=False, penalty=l1)

# Get your LinearSVC model parameterized and trained first 
lsvc = LinearSVC(C=5.0, dual=True, loss='hinge', penalty='l2', tol=0.001)
lsvc2 = CalibratedClassifierCV(lsvc)
lsvc2.fit(X_train, y_train)
pred1 = lsvc2.predict(X_train)
pred1 = pd.DataFrame(data=pred1, columns=['target'])
print(pred1)
y_pred = lsvc2.predict_proba(X_train)
y_pred = pd.DataFrame(data=y_pred, columns=['Pred2', 'Pred']) # Pred is target prediction
y_pred = y_pred.drop(['Pred2'], axis=1)
print(y_pred)
print(y_test)
# Now use the outputs of the LinearSVC model to train the logistic regresion model 
lr = LogisticRegression(C=10.0, dual=False, penalty='l1')
lr.fit(y_pred, y_train)
# After you've fitted the logistic regression model you still must predict on the training data for it 
ytr_pred = lr.predict(y_pred)
y_train2 = pd.DataFrame(data=ytr_pred, columns=['y_pred'])
# Now turn those predictions into predictive probabilities for each team 
ytr_pred_probs = lr.predict_proba(y_train2)
ytr_pred_probs = pd.DataFrame(data=ytr_pred_probs, columns=['Pred2', 'Pred1'])
# Drop the second team which isn't on the Team 1 line anyway 
ytr_pred_probs = ytr_pred_probs.drop(['Pred2'], axis=1)
print(ytr_pred_probs)
# Validate the score of your predictive probabilities against the labeled training data 
lr.score(ytr_pred_probs, y_train)
##################################
# Now let's see how it works against the hold out test data before we decide whether this is worth doing on the final submission file.... ? ? ? 
yt_pred = lsvc2.predict_proba(X_test)
yt_pred = pd.DataFrame(data=yt_pred, columns=['Pred2', 'Pred']) # Pred is target prediction
yt_pred = yt_pred.drop(['Pred2'], axis=1)
# Now use the outputs of the LinearSVC model to train the logistic regression model 
yt_pred2 = lr.predict(yt_pred)
yt_test2 = pd.DataFrame(data=yt_pred2, columns=['y_pred'])
# Now turn those predictions into predictive probabilities for each team 
yt_pred_probs = lr.predict_proba(yt_test2)
yt_pred_probs = pd.DataFrame(data=yt_pred_probs, columns=['Pred2', 'Pred1'])
# Drop the second team which isn't on the Team 1 line anyway 
yt_pred_probs = yt_pred_probs.drop(['Pred2'], axis=1)
print(yt_pred_probs)
# Validate the score of your predictive probabilities against the labeled training data 
lr.score(yt_pred_probs, y_test)


#Generation 5 - Current best internal CV score: -0.5377470801591799
# Best pipeline: LogisticRegression(LogisticRegression(FastICA(input_matrix, tol=0.25), C=25.0, dual=True, penalty=l2), C=15.0, dual=False, penalty=l2)

# 25 Generations # 250 population size  scoring='neg_log_loss', cv=8,
# Generation 25 - Current best internal CV score: -0.5397767537100093
# Best pipeline: LogisticRegression(CombineDFs(input_matrix, CombineDFs(input_matrix, input_matrix)), C=25.0, dual=False, penalty=l2)
#-0.554315389839604



      target
0          1
1          1
2          0
3          1
4          1
5          0
6          0
7          1
8          1
9          0
10         1
11         1
12         1
13         0
14         1
15         1
16         1
17         0
18         1
19         1
20         1
21         0
22         0
23         1
24         1
25         0
26         0
27         0
28         1
29         0
...      ...
1542       1
1543       1
1544       0
1545       1
1546       0
1547       1
1548       0
1549       1
1550       1
1551       0
1552       1
1553       1
1554       0
1555       0
1556       1
1557       1
1558       0
1559       1
1560       1
1561       1
1562       1
1563       1
1564       1
1565       0
1566       0
1567       0
1568       0
1569       1
1570       1
1571       0

[1572 rows x 1 columns]
          Pred
0     0.520368
1     0.695297
2     0.191900
3     0.919545
4     0.509495
5     0.073535
6     0.427578
7     0.613776
8     0.562690
9     0.197339
10  

0.7061068702290076

After After testing the model on our hold out data, we can take the trained model and apply it to this year's NCAA Tournament Submission File that we've created 

In [ ]:
features = ['seed_diff', 'PPG_diff', 'OPPG_diff', 'AOE_diff', 'ADE_diff', 'AEE_diff', 'TOPG_diff', 'FG3_PCT_diff']
ID = st2_tt['ID']
X2 = st2_tt[features]
print(st2_tt.shape)

# Now let's see how it works against the hold out test data before we decide whether this is worth doing on the final submission file.... ? ? ? 
y2_pred = lsvc2.predict_proba(X2)
y2_pred = pd.DataFrame(data=y2_pred, columns=['Pred2', 'Pred']) # Pred is target prediction
y2_pred = y2_pred.drop(['Pred2'], axis=1)
# Now use the outputs of the LinearSVC model to train the logistic regression model 
y2_pred2 = lr.predict(y2_pred)
y2_test2 = pd.DataFrame(data=y2_pred2, columns=['y_pred'])
# Now turn those predictions into predictive probabilities for each team 
y2_pred_probs = lr.predict_proba(y2_test2)
y2_pred_probs = pd.DataFrame(data=y2_pred_probs, columns=['Pred2', 'Pred1'])
# Drop the second team which isn't on the Team 1 line anyway 
y2_pred_probs = y2_pred_probs.drop(['Pred2'], axis=1)
print(y2_pred_probs)

Now let's convert this exercise into the final submission file !

In [ ]:
ID = pd.DataFrame(data=ID, columns=['ID'])
ID['Pred'] = y2_pred_probs['Pred1']
ID.head()
ID[['ID', 'Pred']].to_csv('submission_SVM_LR_Ensemble.csv', index=False)
